# ETL Process

In [1]:
import os 
import glob
import pandas as pd
from sql_queries import *
import psycopg2

In [2]:
# Criando a conexão com o banco 
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=henry password=1234")
curr = conn.cursor()

## Função para encontrar os arquivos json

In [3]:
def get_files(caminho):
    all_data = []
    for pasta , subpasta , arquivo in os.walk(caminho):
        arquivos = glob.glob(os.path.join(pasta, '*.json'))
        for arq in arquivos:
            all_data.append(os.path.abspath(arq))
    return all_data

# Processando song_data

Na primeira etapa do nosso ETL, iremos realizar a carga dos dados song_data para criar as dimensões *songs* e *artistas*

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath  , lines = True)

In [7]:
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AREDL271187FB40F44,NaN,NaN,,Soul Mekanik,SOPEGZN12AB0181B3D,Get Your Head Stuck On Your Neck,45.66159,0


# 1 Song tables

Extraindo os dados ds arquivos 'song_data'

* Selecionando as colunas 'song_id', title , artist_id , year , duration
* Utilizando df.values para selecionar somento os valores do dataframe
* Indexando somente o primeiro registro do dataframe
* Convertendo o array em uma lista

In [8]:
song_data = list(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0])
song_data


['SOPEGZN12AB0181B3D',
 'Get Your Head Stuck On Your Neck',
 'AREDL271187FB40F44',
 0,
 45.66159]

### Inserindo os dados 

Através da query song_table_insert no arquivo sql_queries.py iremos realizar a inserção dos dados dentro de nosso banco por meio da célula abaixo:

In [9]:
curr.execute(song_table_insert, song_data)
conn.commit()

## 2 Tabela artistas

Extraindo dados da tabela artistas:
* Selecionando as colunas: artist_ID, name, location, latitude and longitude
* Utiizando df.values para selecionar somente os valores do dataframe
* Convertendo o array em uma list para insertir na tabela

In [10]:
artista_df = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artista_df

['AREDL271187FB40F44', 'Soul Mekanik', '', nan, nan]

###  Inserindo os dados 

Através da query artist_table_insert no arquivo sql_queries.py iremos realizar a inserção dos dados dentro de nosso banco por meio da célula abaixo:

In [11]:
curr.execute(artist_table_insert , artista_df)
conn.commit()

# Processando log_data

Nesta etapa, o nosso ETL irá realizar a carga dos dados presente nos arquivo log_data para criar as dimensões time e users e também a carga de dados na tabela fato songplays

In [12]:
log_files = get_files('data/log_data/')

In [14]:
filepath = log_files[0]

In [16]:
df = pd.read_json(filepath, lines = True)

In [17]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
1,Jimi Hendrix,Logged In,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Woodstock Inprovisation,200,1543280209796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
2,Building 429,Logged In,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Majesty (LP Version),200,1543280448796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
3,The B-52's,Logged In,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Love Shack,200,1543282396796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38
4,Die Mooskirchner,Logged In,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Frisch und g'sund,200,1543282717796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38


## 3 TIme table

Extraindo os dados para tabela time 
* Filtre o dataframe por NextSong
* Converta a coluna ts (timestamp) para tipo datetime
* Extraia hora, dia, semana, ano, mês e dia da semana e insera os dados em uma lista
* Crie um dataframe unindo os valores os nomes das colunas com a lista de dados

In [18]:
# Filtrando o DF
df = df.loc[df['page'] == 'NextSong']
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88


In [20]:
time  = pd.to_datetime(df['ts'], unit = 'ms')
time.head()

0   2018-11-27 00:52:12.796
1   2018-11-27 00:56:49.796
2   2018-11-27 01:00:48.796
3   2018-11-27 01:33:16.796
4   2018-11-27 01:38:37.796
Name: ts, dtype: datetime64[ns]

In [21]:
colunas = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [31]:
time_data = [df.ts.values, time.dt.hour.values, time.dt.day.values, time.dt.isocalendar().week, time.dt.month.values, time.dt.year.values, time.dt.weekday.values]


In [35]:
time_df = pd.DataFrame(dict(zip(colunas , time_data)))

In [36]:
time_df

,start_time,hour,day,week,month,year,weekday
0,1543279932796,0,27,48,11,2018,1
1,1543280209796,0,27,48,11,2018,1
2,1543280448796,1,27,48,11,2018,1
3,1543282396796,1,27,48,11,2018,1
4,1543282717796,1,27,48,11,2018,1
...,...,...,...,...,...,...,...
298,1543362012796,23,27,48,11,2018,1
299,1543362175796,23,27,48,11,2018,1
300,1543362422796,23,27,48,11,2018,1
301,1543362696796,23,27,48,11,2018,1


### Inserindo os dados

In [42]:
for i , row in time_df.iterrows():
    curr.execute(time_table_insert , list(row))
    conn.commit()
       


# 4 Tabela de usuários

Extraindo os dados da tabela de usuários
* Selecione as colunas user_id, firstName ,lastName, gender, level

In [43]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,88,Mohammad,Rodriguez,M,paid
1,88,Mohammad,Rodriguez,M,paid
2,88,Mohammad,Rodriguez,M,paid
3,38,Gianna,Jones,F,free
4,38,Gianna,Jones,F,free


### Inserindo os dados

In [49]:
for i , row in user_df.iterrows():
    curr.execute(user_table_insert , list(row))
    conn.commit()

#  5 Songplays table

Extraindo os dados da tabela songplays

Esta etapa é um pouco mais complicada que as anteriores, pois a tabela songplays precisa de informações da tabelas de song (song_id) e artistas (artista_id) por isso iremos query nestas tabelas buscando por título , nome do artista e o song duration time.

In [50]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
1,Jimi Hendrix,Logged In,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Woodstock Inprovisation,200,1543280209796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
2,Building 429,Logged In,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Majesty (LP Version),200,1543280448796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
3,The B-52's,Logged In,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Love Shack,200,1543282396796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38
4,Die Mooskirchner,Logged In,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Frisch und g'sund,200,1543282717796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38


In [54]:
for index , row in df.iterrows():
    # Buscando o songid e artist_id a partir das tabelas song e artistas
    
    curr.execute(song_select , (row.song , row.artist , row.length))
    result = curr.fetchone()
    
    
    print(result)
    
    if result:
        songid, artistid = result 
    else:
        songid, artistid = None , None 
        
    # insert songplay record
    songplay_data = [row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    
    curr.execute(songplay_table_insert, songplay_data)
    conn.commit()

    
        
    

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


# Fechando a conexão 

In [55]:
conn.close()

# Implemente o ETL.py

utilize o arquivo ETL.py para inserir todos os dados 